In [1]:
import optuna
import os

import pandas as pd
from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_columns = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_columns", None)  # or 1000
pd.set_option("display.max_rows", None)  # or 1000
pd.set_option("display.max_colwidth", -1)  # or 199

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from warnings import simplefilter

simplefilter(action="ignore", category=DeprecationWarning)

/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_254595/712628210.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)  # or 199


In [4]:
pre = f"copy_tune_2"

env = "bmfl"
alpha = 0.1
dataset = "cifar10"

study_name = f"env_{env}_seed_777_d_{dataset}_alpha_{alpha}_gdt_2"


def f():
    print(f"[+] pre: {pre}")
    print(f"[+] Study: {study_name}")

    storage = f"sqlite:///studys/{pre}/{study_name}.db"
    study = optuna.load_study(study_name=study_name, storage=storage)

    def constraints_func(trial):
        try:
            if trial.params["num_gibbs_draws_train"]:
                return [0]
            if trial.params["num_gibbs_draws_test"]:
                return [0]
        except:
            pass
        return [1]

    # constraints_func = None
    display(optuna.visualization.plot_pareto_front(study, target_names=["best_test_acc", "best_ood_acc"], constraints_func=constraints_func))

    df = study.trials_dataframe()
    df.drop(columns=["datetime_start", "datetime_complete", "duration"], inplace=True)
    df["values_sum"] = df["values_0"] + df["values_1"]
    # df[df.columns].sort_values(by=['values_sum','values_0','values_1'], ascending=[False,False,False]).head(10)
    # df[df.columns].sort_values(by=['values_0','values_1'], ascending=[False,False]).drop_duplicates(subset=["values_0", "values_1"], keep='last').head(10)
    # df[df.columns].sort_values(by=['values_1','values_0'], ascending=[False,False]).drop_duplicates(subset=["values_0", "values_1"], keep='first').head(10)
    df_filtered_test = df[df.columns].sort_values(by=["values_0", "values_1"], ascending=[False, False]).drop_duplicates(subset=["values_0", "values_1"], keep="first")
    df_filtered_ood = df[df.columns].sort_values(by=["values_1", "values_0"], ascending=[False, False]).drop_duplicates(subset=["values_0", "values_1"], keep="first")
    print(f"[+] total trials: {df_filtered_test.shape[0]}")
    display(df_filtered_test.head(6))
    display(df_filtered_ood.head(6))
    display(df[df.columns].sort_values(by=["values_1", "values_0"]))


f()

[+] pre: copy_tune_2
[+] Study: env_bmfl_seed_777_d_cifar10_alpha_0.1_gdt_2


/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_pareto_front.py:227: ExperimentalWarning:

``constraints_func`` argument is an experimental feature. The interface can change in the future.



[+] total trials: 32


,number,values_0,values_1,params_inner_steps,params_lr,params_num_gibbs_draws_test,params_num_gibbs_draws_train,params_wd,state,values_sum
6,6,0.357336,0.401709,5,0.247338,NaN,NaN,0.0008,COMPLETE,0.759045
11,11,0.347178,0.331084,3,0.059622,50.0,50.0,0.0007,COMPLETE,0.678262
30,30,0.344606,0.384615,5,0.031178,NaN,NaN,0.0010,COMPLETE,0.729221
21,21,0.344349,0.321188,5,0.013643,NaN,NaN,0.0009,COMPLETE,0.665536
25,25,0.336762,0.390913,5,0.012340,NaN,NaN,0.0010,COMPLETE,0.727675
23,23,0.334576,0.343230,5,0.015000,NaN,NaN,0.0009,COMPLETE,0.677806


,number,values_0,values_1,params_inner_steps,params_lr,params_num_gibbs_draws_test,params_num_gibbs_draws_train,params_wd,state,values_sum
6,6,0.357336,0.401709,5,0.247338,NaN,NaN,0.0008,COMPLETE,0.759045
26,26,0.295615,0.399010,5,0.010808,NaN,50.0,0.0010,COMPLETE,0.694626
25,25,0.336762,0.390913,5,0.012340,NaN,NaN,0.0010,COMPLETE,0.727675
30,30,0.344606,0.384615,5,0.031178,NaN,NaN,0.0010,COMPLETE,0.729221
29,29,0.325447,0.376968,5,0.033385,50.0,50.0,0.0010,COMPLETE,0.702415
10,10,0.301016,0.371120,1,0.356152,50.0,50.0,0.0006,COMPLETE,0.672136


,number,values_0,values_1,params_inner_steps,params_lr,params_num_gibbs_draws_test,params_num_gibbs_draws_train,params_wd,state,values_sum
4,4,0.149415,0.112011,3,9.933270,NaN,NaN,0.0005,COMPLETE,0.261426
31,31,0.227723,0.206478,5,0.228327,NaN,NaN,0.0010,COMPLETE,0.434201
15,15,0.204706,0.242465,1,3.115311,NaN,NaN,0.0007,COMPLETE,0.447171
8,8,0.206249,0.294197,1,2.975602,NaN,NaN,0.0010,COMPLETE,0.500446
14,14,0.241481,0.302294,1,1.273665,NaN,50.0,0.0007,COMPLETE,0.543775
18,18,0.293172,0.306793,1,0.109045,NaN,NaN,0.0008,COMPLETE,0.599965
9,9,0.285971,0.316689,1,0.356519,NaN,50.0,0.0006,COMPLETE,0.602661
3,3,0.315289,0.318938,3,0.622108,NaN,NaN,0.0006,COMPLETE,0.634227
1,1,0.310788,0.320738,3,0.130304,NaN,NaN,0.0008,COMPLETE,0.631526
21,21,0.344349,0.321188,5,0.013643,NaN,NaN,0.0009,COMPLETE,0.665536


In [5]:
storage = f"sqlite:///studys/{pre}/{study_name}.db"
study = optuna.load_study(study_name=study_name, storage=storage)

In [6]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0])
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[1])

/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



In [7]:
optuna.visualization.plot_contour(study, target=lambda t: sum(t.values)).show("browser")

/gallery_moma/junhyeog.yun/anaconda3/envs/gp-my/lib/python3.9/site-packages/optuna/visualization/_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.

[W 2023-08-26 09:02:46,210] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,211] Param num_gibbs_draws_train unique value length is less than 2.
[W 2023-08-26 09:02:46,213] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,214] Param num_gibbs_draws_train unique value length is less than 2.
[W 2023-08-26 09:02:46,215] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,217] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,218] Param num_gibbs_draws_train unique value length is less than 2.
[W 2023-08-26 09:02:46,218] Param num_gibbs_draws_test unique value length is less than 2.
[W 2023-08-26 09:02:46,219] Param num_gibbs_draws_train uniqu